In [2]:
from pathlib import Path
import SimpleITK as sitk
import numpy as np
%matplotlib notebook
import matplotlib.pyplot as plt

import multiscale.itk.metadata as meta
import multiscale.itk.registration as reg
import multiscale.plotting as myplot
from multiscale.itk.itk_plotting import MultiImageDisplay, RegistrationPointDataAcquisition
import multiscale.itk.itk_plotting as iplot

In [3]:
us_path = Path(r'F:\Research\LINK\Phantom Trials\2018-12-07\Interleaved_5-1V_Sens0-6', 'Reslice of fused_tp_0_ch_0.tif')
mpm_path = Path(r'F:\Research\LINK\Phantom Trials\2018-12-04', 'MPM Grid shrunk 16x in xy.tif')

In [4]:
us_image = sitk.ReadImage(str(us_path))
us_image.SetSpacing([25, 25, 25])
mpm_image = sitk.ReadImage(str(mpm_path))
mpm_image.SetSpacing([16, 16, 7.5])

In [5]:
us_window = myplot.auto_window_level(sitk.GetArrayFromImage(us_image), return_image=False)
mpm_window = myplot.auto_window_level(sitk.GetArrayFromImage(mpm_image), bins=200, upper_limit_fraction=0.3, lower_limit_fraction=0.05, return_image=False)

In [6]:
MultiImageDisplay([mpm_image, us_image], title_list=['MPM', 'US'])

Box(children=(IntSlider(value=0, description='image slice:', max=1), IntSlider(value=20, description='image sl…

FigureCanvasNbAgg()

The US image must be the moving image, as the MPM image is too sparse/too small with zero values for registration to work properly.  At least for the 2 slice example we are working on.

In [6]:
points = RegistrationPointDataAcquisition(mpm_image, us_image)

FigureCanvasNbAgg()

In [7]:
fixed_image_points, moving_image_points = points.get_points_flat()

# Presets
if len(moving_image_points) == 0:
    moving_image_points = [19448.77564964158, 8804.428539426528, 500.0, 19863.14840949821, 5655.195564516129, 500.0, 8757.9584453405, 8224.306675627242, 500.0, 12984.560595878136, 24136.22065412187, 500.0]
if len(fixed_image_points) == 0:
    fixed_image_points = [13164.282365591407, 5833.761577060937, 0.0, 13678.883655913985, 2860.509677419359, 0.0, 2472.0111111111146, 5147.626523297495, 0.0, 5902.68637992832, 21043.088602150536, 0.0]

In [8]:

initial_transform = sitk.LandmarkBasedTransformInitializer(sitk.VersorRigid3DTransform(), 
                                                                fixed_image_points, 
                                                                moving_image_points)
print(initial_transform)

itk::simple::Transform
 VersorRigid3DTransform (0000019252601520)
   RTTI typeinfo:   class itk::VersorRigid3DTransform<double>
   Reference Count: 1
   Modified Time: 1160
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     0.99915 0.0412237 0 
     -0.0412237 0.99915 0 
     0 0 1 
   Offset: [6107.11, 3354.16, 500]
   Center: [8804.47, 8721.25, 0]
   Translation: [6459.14, 2983.79, 500]
   Inverse: 
     0.99915 -0.0412237 0 
     0.0412237 0.99915 0 
     0 0 1 
   Singular: 0
   Versor: [ -0, -0, -0.0206162, 0.999787 ]



In [9]:
initialized_image = sitk.Resample(us_image, mpm_image, initial_transform, sitk.sitkLinear, 0.0, mpm_image.GetPixelID())

In [10]:
MultiImageDisplay([mpm_image, initialized_image], title_list=['MPM', 'US'])

Box(children=(IntSlider(value=0, description='image slice:', max=1), IntSlider(value=0, description='image sli…

FigureCanvasNbAgg()

In [11]:
overlay = iplot.overlay_images(mpm_image, initialized_image, slice=0)
fig, ax = plt.subplots()
ax.imshow(overlay)

FigureCanvasNbAgg()

In [15]:
size = list(mpm_image.GetSize())
size[2] = size[2] + 3
reference_image = sitk.Image(size, mpm_image.GetPixelID())
reference_image.SetOrigin(mpm_image.GetOrigin())
reference_image.SetSpacing(mpm_image.GetSpacing())
reference_image.SetDirection(mpm_image.GetDirection())

mpm_image_extended = sitk.Resample(mpm_image, reference_image, sitk.AffineTransform(3), sitk.sitkLinear, 0.0, mpm_image.GetPixelID())

In [16]:
final_transform, metric, stop = reg.register(mpm_image_extended, us_image, initial_transform=initial_transform)
print(stop)
print(metric)
registered_image = sitk.Resample(us_image, mpm_image, final_transform, sitk.sitkLinear, 0.0, mpm_image.GetPixelID())


RegularStepGradientDescentOptimizerv4: Maximum number of iterations (100) exceeded.
-0.11605432244044712


IndexError: index 20 is out of bounds for axis 0 with size 2

In [17]:

overlay_final = iplot.overlay_images(mpm_image, registered_image, slice=0)
fig2, ax2 = plt.subplots()
ax2.imshow(overlay_final)

FigureCanvasNbAgg()